# Time

As another example of a composite type, we’ll define a `mutable struct` called `MyTime` that records the time of day. The struct definition looks like this:

In [2]:
mutable struct MyTime
    hour :: Int
    minute :: Int
    second :: Int
end

We can create a new `MyTime` object:

In [3]:
time = MyTime(10, 58, 50)

MyTime(10, 58, 50)

Write a function called `print_time` that takes a `Time` object and prints it in the form `"hour:minute:second"`.

In [5]:
function print_time(time)
    @printf("%02d:%02d:%02d", time.hour, time.minute, time.second)
end
print_time(MyTime(1, 7, 2))

01:07:02

Write a boolean function called `is_after` that takes two `Time` objects, `t1` and `t2`, and returns `true` if `t1` follows `t2` chronologically and `false` otherwise. Challenge: don’t use an `if` statement.

In [7]:
function is_after(t1, t2)
    (t1.hour, t1.minute, t1.second) > (t2.hour, t2.minute, t2.second)
end
is_after(MyTime(1, 7, 2+1), MyTime(1, 7, 2))

true

# Pure functions

In the next few sections, we’ll write two functions that add time values. They demonstrate two kinds of functions: *pure functions* and *modifiers*. They also demonstrate a development plan I’ll call *prototype and patch*, which is a way of tackling a complex problem by starting with a simple prototype and incrementally dealing with the complications.

Here is a simple prototype of `add_time`:

In [8]:
function add_time(t1, t2)
    MyTime(t1.hour+t2.hour, t1.minute+t2.minute, t1.second+t2.second)
end

add_time (generic function with 1 method)

The function creates a new `Time` object, initializes its attributes, and returns a reference to the new object. This is called a *pure function* because it does not modify any of the objects passed to it as arguments and it has no effect, like displaying a value or getting user input, other than returning a value.

To test this function, I’ll create two `Time` objects: `start` contains the start time of a movie, like Monty Python and the Holy Grail, and `duration` contains the run time of the movie, which is one hour 35 minutes.

`add_time` figures out when the movie will be done.

In [10]:
start_time = MyTime(11, 45, 0)
duration = MyTime(1, 35, 0)
print_time(add_time(start_time, duration))

12:80:00

The result, 10:80:00 might not be what you were hoping for. The problem is that this function does not deal with cases where the number of seconds or minutes adds up to more than sixty. When that happens, we have to “carry” the extra seconds into the minute column or the extra minutes into the hour column.

Here’s an improved version:

In [11]:
function add_time(t1, t2)
    tsum = MyTime(t1.hour+t2.hour, t1.minute+t2.minute, t1.second+t2.second)
    if tsum.second >= 60
        tsum.second -= 60
        tsum.minute += 1
    end
    if tsum.minute >= 60
        tsum.minute -= 60
        tsum.hour += 1
    end
    tsum
end
print_time(add_time(start_time, duration))

13:20:00

Although this function is correct, it is starting to get big. We will see a shorter alternative later.

# Modifiers

Sometimes it is useful for a function to modify the objects it gets as parameters. In that case, the changes are visible to the caller. Functions that work this way are called *modifiers*.

`increment!`, which adds a given number of seconds to a `Time` object, can be written naturally as a modifier. Here is a rough draft:

In [8]:
function increment!(time, seconds)
    time.second += seconds
    if time.second >= 60
        time.second -= 60
        time.minute += 1
    end
    if time.minute >= 60
        time.minute -= 60
        time.hour += 1
    end
end

increment! (generic function with 1 method)

The first line performs the basic operation; the remainder deals with the special cases we saw before.

Is this function correct? What happens if seconds is much greater than sixty?

In that case, it is not enough to carry once; we have to keep doing it until `time.second` is less than sixty. One solution is to replace the `if` statements with `while` statements. That would make the function correct, but not very efficient.

Anything that can be done with modifiers can also be done with pure functions. In fact, some programming languages only allow pure functions. There is some evidence that programs that use pure functions are faster to develop and less error-prone than programs that use modifiers. But modifiers are convenient at times, and functional programs tend to be less efficient.

In general, I recommend that you write pure functions whenever it is reasonable and resort to modifiers only if there is a compelling advantage. This approach might be called a *functional programming style*.

# Prototyping versus planning

The development plan I am demonstrating is called “prototype and patch”. For each function, I wrote a prototype that performed the basic calculation and then tested it, patching errors along the way.

This approach can be effective, especially if you don’t yet have a deep understanding of the problem. But incremental corrections can generate code that is unnecessarily complicated—since it deals with many special cases—and unreliable—since it is hard to know if you have found all the errors.

An alternative is designed development, in which high-level insight into the problem can make the programming much easier. In this case, the insight is that a `Time` object is really a three-digit number in base 60 (see http://en.wikipedia.org/wiki/Sexagesimal.)! The second attribute is the “ones column”, the minute attribute is the “sixties column”, and the hour attribute is the “thirty-six hundreds column”.

When we wrote `add_time` and `increment!`, we were effectively doing addition in base 60, which is why we had to carry from one column to the next.

This observation suggests another approach to the whole problem—we can convert `Time` objects to integers and take advantage of the fact that the computer knows how to do integer arithmetic.

Here is a function that converts `Time`s to integers:

In [12]:
function time_to_int(time)
    minutes = time.hour * 60 + time.minute
    seconds = minutes * 60 + time.second
end

time_to_int (generic function with 1 method)

And here is a function that converts an integer to a `Time` (recall that `divrm` divides the first argument by the second and returns the quotient and remainder as a tuple):

In [13]:
function int_to_time(seconds)
    (minutes, second) = divrem(seconds, 60)
    hour, minute = divrem(minutes, 60)
    MyTime(hour, minute, second)
end

int_to_time (generic function with 1 method)

You might have to think a bit, and run some tests, to convince yourself that these functions are correct. One way to test them is to check that `time_to_int(int_to_time(x)) == x` for many values of `x`. This is an example of a *consistency check*.

In [14]:
for x in 0:24*60*60
    if time_to_int(int_to_time(x)) ≠ x
        error("Houston, we have a problem!")
    end
end

Once you are convinced they are correct, you can use them to rewrite `add_time`:

In [15]:
function add_time(t1, t2)
    seconds = time_to_int(t1) + time_to_int(t2)
    int_to_time(seconds)
end

add_time (generic function with 1 method)

This version is shorter than the original, and easier to verify. Rewrite increment using `time_to_int` and `int_to_time`.

In [19]:
function increment!(time, seconds)
    println(time_to_int(time))
    int_to_time(time_to_int(time) + seconds)
end

increment!(MyTime(11,16,40), 12870)

40600


MyTime(14, 51, 10)

In some ways, converting from base 60 to base 10 and back is harder than just dealing with times. Base conversion is more abstract; our intuition for dealing with time values is better.

But if we have the insight to treat times as base 60 numbers and make the investment of writing the conversion functions (`time_to_int` and `int_to_time`), we get a program that is shorter, easier to read and debug, and more reliable.

It is also easier to add features later. For example, imagine subtracting two `Time`s to find the duration between them. The naive approach would be to implement subtraction with borrowing. Using the conversion functions would be easier and more likely to be correct.

Ironically, sometimes making a problem harder (or more general) makes it easier (because there are fewer special cases and fewer opportunities for error).

# Debugging

A `Time` object is well-formed if the values of minute and second are between 0 and 60 (including 0 but not 60) and if hour is positive. hour and minute should be integral values, but we might allow second to have a fraction part.

Requirements like these are called *invariants* because they should always be true. To put it a different way, if they are not true, something has gone wrong.

Writing code to check invariants can help detect errors and find their causes. For example, you might have a function like `valid_time` that takes a `Time` object and returns `false` if it violates an invariant:

In [14]:
function valid_time(time)
    if time.hour < 0 || time.minute < 0 || time.second < 0
        return false
    end
    if time.minute >= 60 || time.second >= 60
        return false
    end
    true
end

valid_time (generic function with 1 method)

At the beginning of each function you could check the arguments to make sure they are valid:

In [17]:
function add_time(t1, t2)
    if !valid_time(t1) || !valid_time(t2)
        error("invalid MyTime object in add_time")
    end
    seconds = time_to_int(t1) + time_to_int(t2)
    int_to_time(seconds)
end
add_time(MyTime(1, 1, 1), MyTime(1, 61, 1))

LoadError: [91minvalid MyTime object in add_time[39m

Or you could use an `@assert` macro, which checks a given invariant and raises an exception if it fails:

In [20]:
function add_time(t1, t2)
    @assert(valid_time(t1) && valid_time(t2), "invalid MyTime object in add_time")
    seconds = time_to_int(t1) + time_to_int(t2)
    int_to_time(seconds)
end
add_time(MyTime(1, 1, 1), MyTime(1, 61, 1))

LoadError: [91mUndefVarError: valid_time not defined[39m

`@assert` macros are useful because they distinguish code that deals with normal conditions from code that checks for errors.